# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api
from Clust.clust.ML.tool import meta

influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

cpu is available.


# 1. set param from Front End

In [2]:
modelName = 'anomaly_detection_SWaT_anomaly_detection_BeatGAN_ad_'
params1 = {
    'ingestion_param_X': {
        'bucket_name': 'local',
        'ms_name': 'anomaly_detection_SWaT',
        'feature_list': 'None'
    },
    'ingestion_param_y': {
        'bucket_name': 'local',
        'ms_name': 'anomaly_detection_SWaT',
        'feature_list': 'None'
    },
    'data_y_flag': 'None',
    'model_name': modelName
}

# params1 = {
#     "ingestion_param_X" :{
#         "bucket_name": 'integration',
#         "ms_name" : 'regression_energy_cleanLevel4_testX',
#         "feature_list": "None"
#     },
#     "ingestion_param_y":{
#         "bucket_name": 'integration',
#         "ms_name" : 'regression_energy_cleanLevel4_testy',
#         "feature_list": "None"
#     },
#     'data_y_flag' : "None",
#     'model_name':'regression_energy_cleanLevel4_trainX_regression_GRU_rg_'
# }


# 2. Backend

In [3]:
# 1. Backend Parameter Setting
params = params1

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [4]:
# 2. get model meta
model_name = params['model_name']
meta_file_name = "./meta.json"
# for testing mongodb model
# model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
# for testing local model
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)

# data parameters from model meta
params['ingestion_param_X']['feature_list'] = model_meta['ingestion_param_X']['feature_list']
params['ingestion_param_y']['feature_list'] = model_meta['ingestion_param_y']['feature_list']
params['data_y_flag'] = model_meta['data_y_flag']

# scaler/transform info for test data preparation
params['scaler_param'] = model_meta['scaler_param']
params['transform_param'] = model_meta['transform_param']

# model info for ML test
params['model_info'] = model_meta['model_info']

In [5]:
# for data preparation (temp)
def pre_test_df(test_df):
    test_df['attack'] = test_df['attack'].apply(lambda x : 1 if x > 0 else 0)
    split_index = int(len(test_df) * 0.5)
    
    valid_x = test_df.iloc[:split_index,:].drop(columns=['timestamp','attack'])
    valid_y = test_df.iloc[:split_index,:]['attack']

    test_x = test_df.iloc[split_index:,:].drop(columns=['timestamp','attack'])
    test_y = test_df.iloc[split_index:,:]['attack']
    
    return valid_x, valid_y, test_x, test_y

# 3. Data Preparation
# test_X_array, test_y_array, scaler_X, scaler_y = AD_api.test_data_preparation(params, influxdb_client)

# 추후 ML_api 의 데이터 준비 형식으로 대체 가능
import pandas as pd

# from test.csv
# 보통 train set 에서 train/val 나누지 않는지?
test_df = pd.read_csv('./data/test_ver2.csv')
_, _, test_X, test_y = pre_test_df(test_df)

In [6]:
print(type(test_X))
print(test_X.shape)  # TBD: [seq_len, n_features]

<class 'pandas.core.frame.DataFrame'>
(22496, 51)


In [7]:
# set scaler by data y flag
# if params['data_y_flag']:
#     scaler = scaler_y
# else:
#     scaler = scaler_X

In [8]:
# model test
# value: attack prediction: score?
result = ML_api.ML_test(params, test_X, test_y, scaler=None)

features shape: (22496, 51) targets shape:  (22496,)

Start testing data

Test inference start


  0%|                                                                                                                                                                                                                                                                       | 0/693 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 693/693 [00:31<00:00, 22.32it/s]


False
=========================data
       value  prediction
0          1    0.694053
1          1    0.694035
2          1    0.694534
3          1    0.694346
4          1    0.694075
...      ...         ...
22171      0    0.709731
22172      0    0.711013
22173      0    0.709989
22174      0    0.711019
22175      0    0.709554

[22176 rows x 2 columns]


In [9]:
result['result_metrics']

{'accuracy': 0.330537518037518,
 'recall': 1.0,
 'precision': 0.330537518037518,
 'f_score': 0.4968480986917915}